[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://raw.githubusercontent.com/adugnag/S1-S2_Transformer/main/Prepare_data.ipynb?token=GHSAT0AAAAAABNICAQHDTK3HNFP2CPI742OYZEK3GQ)

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:

import tensorflow as tf
print(tf.__version__)

In [ ]:
!pip install -q -U keras-tuner

import keras_tuner as kt

In [ ]:
import folium
print(folium.__version__)

In [ ]:
!wget https://raw.githubusercontent.com/adugnag/S1-S2_Transformer/main/helper.py

from helper import model_builder, get_dataset

In [ ]:
params = {'DATA_BUCKET' : 'senalerts_dl4',
          'OUTPUT_BUCKET' : 'senalerts_dl4',
          'FOLDER' : 'TDF1',
          'BANDS1' :['VV','VH'],
          'BANDS2' : ['NBR'],
          'TRAIN_FILE_PREFIX' : 'Training_TDF_BOL_S1_desc_S2_v2',
          'TEST_FILE_PREFIX' : 'Testing_TDF_BOL_S1_desc_S2_v2',
          'VALIDATION_FILE_PREFIX' :'Validation_TDF_BOL_S1_desc_S2_v2',
          'LABEL':'Label',
          'NUM_S1':25,
          'NUM_S2':25,
          'MODEL_NAME':'TDF_MHSA_PAR_S1_S2_v2'}

In [ ]:
# Sentinel-1 features
FEATURE_NAMES1 = list(params['BANDS1'])
FEATURE_NAMES1.append(params['LABEL'])

# List of fixed-length features
columns1 = [
  tf.io.FixedLenFeature(shape=[params['NUM_S1']], dtype=tf.float32),
  tf.io.FixedLenFeature(shape=[params['NUM_S2']], dtype=tf.float32),
  tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
]

# Dictionary with feature names as keys, fixed-length features as values.
FEATURES_DICT1 = dict(zip(FEATURE_NAMES1, columns1))

#Sentnel-2 features

FEATURE_NAMES2 = list(params['BANDS2'])

columns2 = [
  tf.io.FixedLenFeature(shape=[25], dtype=tf.float32) for k in FEATURE_NAMES2
]

FEATURES_DICT2 = dict(zip(FEATURE_NAMES2, columns2))

In [ ]:
#File path

TRAIN_FILE_PATH = 'gs://' + params['OUTPUT_BUCKET'] + '/' + params['FOLDER'] + '/' + params['TRAIN_FILE_PREFIX'] + '.tfrecord.gz'
TEST_FILE_PATH = 'gs://' + params['OUTPUT_BUCKET'] + '/' + params['FOLDER'] + '/'  + params['TEST_FILE_PREFIX'] + '.tfrecord.gz'
VALIDATION_FILE_PATH = 'gs://' + params['OUTPUT_BUCKET'] + '/' + params['FOLDER']+ '/'  + params['VALIDATION_FILE_PREFIX'] + '.tfrecord.gz'

#get training, validation and test dataset
train_dataset, validation_dataset, test_dataset = get_dataset(TRAIN_FILE_PATH, TEST_FILE_PATH, VALIDATION_FILE_PATH, FEATURES_DICT1, FEATURES_DICT2, batch_size=16)

# Print the first parsed 

from pprint import pprint
pprint(iter(train_dataset).next())

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     overwrite=True)

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)

In [ ]:
tuner.search(train_dataset, epochs=50, validation_data=validation_dataset, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the dense
layer is {best_hps.get('filters')}, The optimal number of units in the LSTM
layer is {best_hps.get('lstm units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)

history = model.fit(train_dataset, epochs=50,validation_data=validation_dataset)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model with the best epoch
hypermodel.fit(x=train_dataset, validation_data = validation_dataset, epochs=best_epoch)

In [ ]:
#evaluate model
eval_result = hypermodel.evaluate(test_dataset)
print("[test loss, test accuracy]:", eval_result)

In [ ]:
MODEL_DIR = 'gs://' + params['OUTPUT_BUCKET'] + '/' + params['FOLDER'] + '/' + params['MODEL_NAME']
hypermodel.save(MODEL_DIR, save_format='tf')